In [1]:
import requests
from bs4 import BeautifulSoup
import math
import json
from tqdm import tqdm

In [2]:
SIZE = 60
INIT_PAGE = 1
CATEGORY = 'made_in_kz'
BASE_URL = f"https://www.lamoda.kz/a/29512/{CATEGORY}/?sitelink=topmenuW&l=10&page={INIT_PAGE}"

In [3]:
res = requests.get(BASE_URL)

In [4]:
soup = BeautifulSoup(res.content)
amountOfRes = list(soup.findAll("span", {"class": "products-catalog__head-counter"}))[0].text

In [5]:
def parse_page(soup):
    dict_publ = dict()
    links = soup.findAll("a", {"class": "products-list-item__link"})
    for link in links:
        try:
            featured_links = link['href'].split("/");
            if ('p' == featured_links[1]):
                dict_publ[featured_links[2]] = link['href']
        except:
            pass
    return dict_publ

In [6]:
amount = int(amountOfRes.split(" ")[0])
pages_amount = math.ceil(amount/SIZE)

In [7]:
papers = parse_page(soup)
# print(papers)

In [8]:
for strt in range(1, pages_amount+1):
    BASE_URL = f"https://www.lamoda.kz/a/29512/{CATEGORY}/?sitelink=topmenuW&l=10&page={strt}"
    res = requests.get(BASE_URL)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content)
        buff = parse_page(soup)
        papers.update(parse_page(soup))
    else: 
        print(res.status_code, " " , BASE_URL);

In [9]:
with open(f"{CATEGORY}.json", "w") as fh:
    json.dump(papers, fh, indent = 2)

In [10]:
for paper in list(papers.items())[:1]:
    paper_url = "https://www.lamoda.kz"+paper[1]
    res = requests.get(paper_url)
    soup = BeautifulSoup(res.content)

In [11]:
list(soup.select(".product-title__model-name"))[0].text

'Блуза '

In [18]:
list(soup.select(".product-title__brand-name"))[0].text

'Блуза Sana.moda Блуза '

In [13]:
list(soup.select("#vue-root div.product-prices span"))[0].text

'22 000 ₸'

In [44]:
len(papers)

466

In [53]:
def get_items_data():
    data_dict = dict()
    for paper in list(papers.items())[:200]:
        paper_url = "https://www.lamoda.kz"+paper[1]
        res = requests.get(paper_url)
        soup = BeautifulSoup(res.content)
        leftSide = list(soup.select(".product-title__model-name"))[0].text + list(soup.select(".product-title__brand-name"))[0].text
        rightSide = list(soup.select("#vue-root div.product-prices span"))[0].text
        
        data_dict[leftSide]= rightSide

    return data_dict
    

In [54]:
items_info = get_items_data()

In [55]:
print(len(items_info))

115


In [56]:
with open(f"{CATEGORY}_items_info.json", "w", encoding='utf-8') as fh:
    json.dump(items_info, fh, indent = 2, ensure_ascii=False)